In [71]:
import pandas as pd
from scapy.all import PcapReader
import os 
from datetime import datetime 
import numpy as np
from tqdm import tqdm

from scapy.layers.dot11 import Dot11

In [35]:
def read_pcapng(filename):
    df = pd.DataFrame(columns=['Timestamp', 'Sender', 'Receiver', 'Signal-Strength'])

    for pkt in PcapReader(filename):
        if pkt.haslayer('Dot11'):
            timestamp = datetime.fromtimestamp(float(pkt.time))
            rssi =  pkt.getlayer('RadioTap').dBm_AntSignal
            sender =  pkt.addr2
            receiver =  pkt.addr1
        else:
            print('this package is not a wifi package')
            continue

        new_row = pd.DataFrame({'Timestamp': [timestamp], 'Sender': [sender], 'Receiver': [receiver], 'Signal-Strength': [rssi]})
        df = pd.concat([df, new_row], ignore_index=True)

    return df


In [36]:
filepath = os.getcwd()
# print(filepath)
filepath += "\\Lecture-Data\\Lecture-12-12_00002_20231212082631.pcapng"

df = read_pcapng(filepath)

print(df)


C:\Users\Philipp\AppData\Local\Temp\ipykernel_13024\1954771597.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)


                      Timestamp             Sender           Receiver  \
0    2023-12-12 08:26:31.261669  ea:f5:71:1e:46:bc  00:f6:63:ad:8a:a0   
1    2023-12-12 08:26:31.294627  ea:f5:71:1e:46:bc  00:f6:63:ad:8a:a0   
2    2023-12-12 08:26:31.879087  ea:f5:71:1e:46:bc  00:f6:63:ad:8a:a0   
3    2023-12-12 08:26:31.897903               None  ea:f5:71:1e:46:bc   
4    2023-12-12 08:26:32.782836  3a:cf:d1:b9:84:d6  ff:ff:ff:ff:ff:ff   
...                         ...                ...                ...   
2425 2023-12-12 08:36:23.525919               None  ac:bd:70:d8:71:cc   
2426 2023-12-12 08:36:25.542764               None  26:3b:22:93:ac:e6   
2427 2023-12-12 08:36:26.894979  16:cf:27:01:cb:7b  ff:ff:ff:ff:ff:ff   
2428 2023-12-12 08:36:26.975547  16:cf:27:01:cb:7b  ff:ff:ff:ff:ff:ff   
2429 2023-12-12 08:36:26.995736  16:cf:27:01:cb:7b  ff:ff:ff:ff:ff:ff   

     Signal-Strength  
0                -81  
1                -83  
2                -84  
3                -63  
4       

In [37]:
addresses = pd.concat((df['Sender'].astype(str), df['Receiver'].astype(str)))
addresses = addresses.drop(addresses[addresses == 'None'].index)

print(addresses.nunique())

474


In [46]:
unique_addresses = addresses.unique()
addresses_counts = np.zeros(addresses.nunique())
for index, address in enumerate(unique_addresses):
    addresses_counts[index] = addresses[addresses == address].count()

print(addresses_counts)

[3.000e+00 2.000e+00 2.000e+00 1.000e+00 6.000e+00 2.000e+00 3.000e+00
 2.000e+00 2.000e+00 1.000e+00 2.000e+00 9.400e+01 2.000e+00 1.000e+00
 3.000e+00 1.000e+00 2.000e+00 2.000e+00 2.000e+00 2.000e+00 1.000e+00
 1.200e+01 1.000e+00 2.000e+00 1.000e+00 1.000e+00 1.000e+00 2.000e+00
 1.000e+00 1.000e+00 2.000e+00 1.000e+00 2.000e+00 1.200e+01 2.000e+00
 1.000e+00 1.000e+00 2.000e+00 1.000e+01 1.000e+01 1.000e+00 1.000e+00
 2.000e+00 2.000e+00 1.000e+00 1.000e+00 2.000e+00 4.000e+00 1.000e+00
 2.000e+00 2.000e+00 6.000e+00 2.000e+00 2.000e+00 1.000e+00 8.000e+00
 4.000e+00 1.000e+00 2.000e+00 1.000e+00 2.000e+00 2.000e+00 4.000e+00
 4.000e+00 7.000e+00 1.000e+00 7.000e+00 2.000e+00 1.000e+00 1.000e+00
 1.000e+00 7.400e+01 3.000e+00 3.000e+00 4.000e+00 2.000e+00 1.000e+00
 1.000e+00 1.000e+00 3.000e+00 6.400e+01 3.000e+00 1.000e+00 2.000e+00
 1.000e+00 2.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00
 1.000e+00 2.000e+00 1.000e+00 1.420e+02 1.400e+01 1.000e+00 1.000e+00
 2.000

In [47]:
import glob

filepath = os.getcwd()
# print(filepath)
filepath += "\\Lecture-Data"

def read_multiple_pcapng(directory):
   # Define the path to the directory containing the pcapng files
   path = os.path.join(directory, "*.pcapng")
   
   # Identify all pcapng files in the directory
   all_files = glob.glob(path)
   
   # Initialize an empty DataFrame
   df = pd.DataFrame(columns=['Timestamp', 'Sender', 'Receiver', 'Signal-Strength'])
   
   # Iterate over all pcapng files
   for filename in all_files:
       # Read the pcapng file and append the result to the DataFrame
       df = pd.concat([df, read_pcapng(filename)], ignore_index=True)
   
   return df

# Call the function
df = read_multiple_pcapng(filepath)

print(df)


C:\Users\Philipp\AppData\Local\Temp\ipykernel_13024\1954771597.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)
C:\Users\Philipp\AppData\Local\Temp\ipykernel_13024\3338776499.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, read_pcapng(filename)], ignore_index=True)
C:\Users\Philipp\AppData\Local\Temp\ipykernel_13024\1954771597.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is

                       Timestamp             Sender           Receiver  \
0     2023-12-12 08:26:31.261669  ea:f5:71:1e:46:bc  00:f6:63:ad:8a:a0   
1     2023-12-12 08:26:31.294627  ea:f5:71:1e:46:bc  00:f6:63:ad:8a:a0   
2     2023-12-12 08:26:31.879087  ea:f5:71:1e:46:bc  00:f6:63:ad:8a:a0   
3     2023-12-12 08:26:31.897903               None  ea:f5:71:1e:46:bc   
4     2023-12-12 08:26:32.782836  3a:cf:d1:b9:84:d6  ff:ff:ff:ff:ff:ff   
...                          ...                ...                ...   
99888 2023-12-12 08:15:03.673286               None  d0:16:b4:fc:98:c3   
99889 2023-12-12 08:15:03.802218  de:38:e5:24:8d:9d  ff:ff:ff:ff:ff:ff   
99890 2023-12-12 08:15:05.086811  6e:36:4c:8b:83:ab  ff:ff:ff:ff:ff:ff   
99891 2023-12-12 08:15:05.599234  fa:c6:91:03:73:0b  ff:ff:ff:ff:ff:ff   
99892 2023-12-12 08:15:05.621162  fa:c6:91:03:73:0b  ff:ff:ff:ff:ff:ff   

      Signal-Strength  
0                 -81  
1                 -83  
2                 -84  
3              

In [48]:
addresses = pd.concat((df['Sender'].astype(str), df['Receiver'].astype(str)))

print(addresses.nunique())
print(df['Sender'].nunique())
print(df['Receiver'].nunique())

12073
11764
842


In [67]:
unique_addresses = addresses.unique()
addresses_counts = np.zeros((addresses.nunique(), 2))

In [73]:
for index, address in tqdm(enumerate(unique_addresses), total=addresses.nunique()):
    indices = addresses[addresses == address].index
    addresses_counts[index,1] = indices[0]
    addresses_counts[index,0] = indices[-1]


print(addresses_counts)

100%|██████████| 12073/12073 [02:50<00:00, 70.77it/s]


[[9.9053e+04 0.0000e+00]
 [9.9888e+04 3.0000e+00]
 [5.0000e+00 4.0000e+00]
 ...
 [9.8632e+04 9.8603e+04]
 [9.9376e+04 9.9334e+04]
 [9.9431e+04 9.9431e+04]]
